<a href="https://colab.research.google.com/github/ashishvanguri/NNDL/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_excel(os.path.join(os.getcwd(), 'Assets', '/content/drive/MyDrive/Load Data in kW.xlsx'))
print(data.shape)
data.head()

(2184, 3)


,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [ ]:
x = []
y = []
for i in range(24 * 7, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i - 24 * 7]) #Since x is 24 * 7 hours before y
    y.append(data['Load (kW)'].iloc[i])

In [ ]:
data = pd.DataFrame({'Previous Week Load' : x, 'Present Week Load' : y})
data.head()

,Previous Week Load,Present Week Load
0,5551.82208,5346.52812
1,4983.17184,5138.89596
2,4888.39680,4940.77248
3,5072.95872,4766.18688
4,5196.25980,5020.11540


In [ ]:
maxX = np.max(data['Previous Week Load'])
minX = np.min(data['Previous Week Load'])
maxY = np.max(data['Present Week Load'])
minY = np.min(data['Present Week Load'])

data['Previous Week Load'] = (data['Previous Week Load'] - minX) / (maxX - minX)
data['Present Week Load'] = (data['Present Week Load'] - minY) / (maxY - minY)

In [ ]:
x = data['Previous Week Load']
y = data['Present Week Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [ ]:
m = np.random.uniform(-2, 2)
c = np.random.uniform(-2, 2)
eta = 0.5
epochs = 500
gm2 = 0
gc2 = 0
e = 1E-6
for _ in range(epochs):
    for i in range(trainX.shape[0]):
        gradM = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c) * trainX.iloc[i]
        gradC = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c)
        gm2 += (gradM ** 2)
        gc2 += (gradC ** 2)
        m -= (eta * gradM) / ((e + gm2) ** 0.5)
        c -= (eta * gradC) / ((e + gc2) ** 0.5)
print(f'm = {m}\nc = {c}')

m = 0.485735245010263
c = 0.23135549380743772


In [ ]:
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Training Error :
MAE : 737.0271347472773
MSE : 891424.7197333745
RMSE : 944.1529112031454
Testing Error :
MAE : 787.1171770619974
MSE : 968116.1927509429
RMSE : 983.9289571665948


In [ ]:
pd.DataFrame({'Actual' : trainY, 'Predicted' : train_pred})

,Actual,Predicted
0,5086.98792,5060.572102
1,6378.60960,5731.518611
2,7647.78456,6182.149803
3,6165.05400,5656.059857
4,5918.45184,5501.808830
...,...,...
1809,5020.11540,5866.677664
1810,4048.82712,5524.158562
1811,5674.03200,5578.176726
1812,5621.65632,6995.528497


In [ ]:
pd.DataFrame({'Actual' : testY, 'Predicted' : test_pred})

,Actual,Predicted
0,7609.43808,6756.045445
1,4187.24856,5748.792301
2,6738.69240,6181.543708
3,5640.67368,6206.090532
4,6550.07760,5442.411578
...,...,...
197,5761.79244,6598.839708
198,6955.67736,6087.144504
199,8607.69360,6021.080214
200,6532.93080,6219.500371


In [ ]:
x = float(input('Enter the load at same hour in previous week: '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load at present hour :', prediction)

Enter the load at same hour in previous week: 4500
Predicted load at present hour : 5185.757892059041
